##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - Prompt Chaining and Iterative Generation
This notebook demonstrates how to use prompt chaining and iterative generation with Gemma through a story writing example.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Prompt_chaining.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Gemma setup

To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run
  the Gemma 2B model.
* Generate and configure a Kaggle username and an API key as Colab secrets.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.


### Configure your credentials

Add your your Kaggle credentials to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create new secrets: `KAGGLE_USERNAME` and `KAGGLE_KEY`
3. Copy/paste your username into `KAGGLE_USERNAME`
3. Copy/paste your key into `KAGGLE_KEY`
4. Toggle the buttons on the left to allow notebook access to the secrets.


In [2]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

### Install dependencies
Run the cell below to install all the required dependencies.

In [3]:
!pip install -q -U tensorflow keras keras-nlp

## Prompt chaining

Prompt chaining is a powerful technique for managing complex tasks that are difficult to accomplish in a single step.

It entails breaking a large task into smaller, linked prompts, where each prompt's output feeds into the next. This step-by-step method steers the language model through the process. Key advantages include:



*   Enhanced accuracy: Focused, smaller prompts produce better results from the language model.
*   Easier debugging: Pinpointing and fixing issues within the chain is straightforward, allowing for precise improvements.
* Handling complexity: Dividing intricate problems into manageable steps enables the language model to address more complex tasks.


## Iterative Generation
Iterative generation is the process of creating the desired output step by step. This method is particularly useful for writing stories that exceed the length limitations of a single generation window. The advantages of iterative generation include:

* Extended outputs: It enables the production of longer and more detailed content, going beyond the constraints of a single generation window.
* Enhanced flexibility: Adjustments and refinements can be made at each
iteration, ensuring the story progresses as intended.
* Human oversight: Feedback and guidance can be provided at each step, ensuring the story stays true to the creator's vision.


By using **prompt chaining** and **iterative generation** together, you can create an interesting and well-structured story, adding to it piece by piece, while still having control over how it unfolds.

### Gemma

**About Gemma**

Gemma is a family of lightweight, state-of-the-art open models from Google, built from the same research and technology used to create the Gemini models. They are text-to-text, decoder-only large language models, available in English, with open weights, pre-trained variants, and instruction-tuned variants. Gemma models are well-suited for a variety of text generation tasks, including question answering, summarization, and reasoning. Their relatively small size makes it possible to deploy them in environments with limited resources such as a laptop, desktop or your own cloud infrastructure, democratizing access to state of the art AI models and helping foster innovation for everyone.

Here's the [official documentation](https://ai.google.dev/gemma/docs/formatting) regarding promping instruction-tuned models.

In [4]:
import keras
import keras_nlp
from IPython.display import display, Markdown, Latex

In [5]:
# Let's load Gemma using Keras
gemma_model_id = "gemma_1.1_instruct_2b_en"
gemma = keras_nlp.models.GemmaCausalLM.from_preset(gemma_model_id)

Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metada

## Story writing baseline: persona, premise, outline

As the first step, we'll create a persona, which the LLM should take to perform the task. In this example, we want it to act like a children's author, who wants to write a new funny and educating story.

In [6]:
persona = """You are a children's author with a penchant for humorous, yet educating stories.
Your ultimate goal is to write a new story to be published in a children magazine."""

The upcoming subtask for the LLM is to develop a premise for the story. To achieve this, we require a prompt. We'll use the persona description to generate the premise prompt. In this example, we want the model to create a story about bunnies.

In [7]:
premise_prompt = f"""<start_of_turn>user
{persona}

Write a single-sentence premise for a children's story featuring bunnies.<end_of_turn>
<start_of_turn>model\n"""

In [8]:
premise_response = gemma.generate(premise_prompt, max_length=512)
premise = premise_response[len(premise_prompt) :]
display(Markdown(premise))

In the bustling meadow of Whispering Woods, a clumsy bunny named Pip learns that even the smallest of creatures can achieve extraordinary things when they work together.

We'll use the generated premise to create a prompt for the next step, which will produce the story outline.



In [9]:
outline_prompt = f"""<start_of_turn>user
{persona}

You have a gripping premise in mind:

{{premise}}

Create an outline for your story's plot consisting of 5 key points.<end_of_turn>
<start_of_turn>model\n"""

In [10]:
full_outline_prompt = outline_prompt.format(premise=premise)
outline_response = gemma.generate(full_outline_prompt, max_length=512)
outline = outline_response[len(full_outline_prompt) :]
display(Markdown(outline))

## Plot Outline:

**1. The Clumsy Bunny:**
- Pip, a timid bunny, struggles with his clumsy nature.
- He dreams of doing something extraordinary but lacks the confidence to try.

**2. The Whisper of Teamwork:**
- Pip encounters a timid squirrel who inspires him to believe in his abilities.
- Together, they discover the magic of working as a team.

**3. The Power of Small Acts:**
- Pip and the squirrel embark on small, seemingly insignificant tasks around the meadow.
- These tasks gradually build confidence and a sense of accomplishment.

**4. Overcoming Fear:**
- Pip faces his biggest fear, which holds him back from achieving his dreams.
- Through the support of his newfound friends, he learns to overcome his fear and embrace his potential.

**5. The Celebration of Togetherness:**
- Pip and the squirrel celebrate their achievements as a team.
- They realize that even the smallest of creatures can achieve extraordinary things when they work together.

Once we have the plan, we'd like Gemma to begin writing the story. In the prompt, include all the necessary information we've gathered so far: the persona, premise, and outline.


In [11]:
starting_prompt = f"""<start_of_turn>user
{persona}

You have a gripping premise in mind:

{{premise}}

Your imagination has crafted a narrative outline:

{{outline}}

First, silently review the outline and the premise. Consider how to start the
story.

Your task is to write a part of the story that covers only the first point of the outline.
You are not expected to finish the whole story now.
Do not write about the next points, only the first one plot point!!!

Try to write 10 sentences.
Remember, DO NOT WRITE A WHOLE STORY RIGHT NOW.<end_of_turn>
<start_of_turn>model\n"""

In [12]:
full_starting_prompt = starting_prompt.format(premise=premise, outline=outline)
starting_response = gemma.generate(full_starting_prompt, max_length=1000)
draft = starting_response[len(full_starting_prompt) :]
display(Markdown(draft))

Pip, a bunny with fur as white as freshly fallen snow, sat nestled beneath a towering oak, his eyes fixed on the path ahead. His heart thumped a frantic rhythm against his ribs, a counterpoint to the gentle rustling of leaves in the wind. He dreamt of soaring through the sky, of his name echoing through the vast meadow, but fear held him captive.

The memory of his clumsy attempts to hop and skip always surfaced, tripping him over pebbles and sending him sprawling in a heap. He longed to be like the other bunnies, agile and graceful, but his clumsy nature held him captive.

If you're pleased with the start of your story, you can continue it by further prompting the model with the text written so far. You can also add guidelines to help the model write appropriately and avoid concluding the story too quickly.

In [13]:
guidelines = """Writing Guidelines

Remember, your main goal is to write as much as you can. If you get through
the story too fast, that is bad. Expand, never summarize. Don't repeat previous
parts of the story, only expand."""


continuation_prompt = f"""<start_of_turn>user
{persona}

You have a following premise in mind:

{{premise}}

The outline of the story looks like this:

{{outline}}


Here's what you've written so far:

{{story_text}}


=====
First, silently review the premise, the outline and the story you've written so far.

Write the continuation - the next 5 sentences that cover the next outline point. Stick to the outline.

However, once the story is COMPLETELY finished, write IAMDONE.

{guidelines}<end_of_turn>
<start_of_turn>model\n"""

In [14]:
full_continuation_prompt = continuation_prompt.format(
    premise=premise, outline=outline, story_text=draft
)
continuation_response = gemma.generate(full_continuation_prompt, max_length=1000)
continuation = continuation_response[len(full_continuation_prompt) :]
display(Markdown(continuation))

Pip sighed, his whiskers twitching in frustration. He hopped closer to the edge of the meadow, his eyes scanning the vast expanse. A sense of longing gnawed at him, a yearning to do something extraordinary, something that would make him truly shine.

Suddenly, a flicker of inspiration struck him. He remembered the squirrel he had seen earlier, timid and hesitant, but radiating an aura of quiet confidence. A smile crept across Pip's face. He knew what he had to do.

With newfound determination, Pip hopped towards the squirrel, his tiny paws extending a gentle touch towards his friend. "Let's do this," he whispered, his voice barely above a whisper.

Together, Pip and the squirrel began their small but mighty tasks. Pip carefully placed pebbles to create a path for the squirrel, while the squirrel, with her nimble leaps, helped him reach higher branches.

As they worked side by side, the meadow came alive with their collaborative efforts. The sun shone brighter, the wind rustled the leaves, and the air buzzed with a sense of purpose.

Pip and the squirrel, their bond stronger than ever, realized that even the smallest of creatures could achieve extraordinary things when they worked together. And so, their tale of teamwork was whispered throughout the meadow, inspiring other timid creatures to believe in their own potential.

Add the continuation to the initial draft, keep building the story iteratively, until 'IAMDONE' is seen

In [15]:
draft = draft + "\n\n" + continuation

while "IAMDONE" not in continuation:
    full_continuation_prompt = continuation_prompt.format(
        premise=premise, outline=outline, story_text=draft
    )
    continuation_response = gemma.generate(full_continuation_prompt, max_length=5000)
    continuation = continuation_response[len(full_continuation_prompt) :]
    draft = draft + "\n\n" + continuation

# Remove 'IAMDONE' and print the final story
final = draft.replace("IAMDONE", "").strip()
display(Markdown(final))

Pip, a bunny with fur as white as freshly fallen snow, sat nestled beneath a towering oak, his eyes fixed on the path ahead. His heart thumped a frantic rhythm against his ribs, a counterpoint to the gentle rustling of leaves in the wind. He dreamt of soaring through the sky, of his name echoing through the vast meadow, but fear held him captive.

The memory of his clumsy attempts to hop and skip always surfaced, tripping him over pebbles and sending him sprawling in a heap. He longed to be like the other bunnies, agile and graceful, but his clumsy nature held him captive.

Pip sighed, his whiskers twitching in frustration. He hopped closer to the edge of the meadow, his eyes scanning the vast expanse. A sense of longing gnawed at him, a yearning to do something extraordinary, something that would make him truly shine.

Suddenly, a flicker of inspiration struck him. He remembered the squirrel he had seen earlier, timid and hesitant, but radiating an aura of quiet confidence. A smile crept across Pip's face. He knew what he had to do.

With newfound determination, Pip hopped towards the squirrel, his tiny paws extending a gentle touch towards his friend. "Let's do this," he whispered, his voice barely above a whisper.

Together, Pip and the squirrel began their small but mighty tasks. Pip carefully placed pebbles to create a path for the squirrel, while the squirrel, with her nimble leaps, helped him reach higher branches.

As they worked side by side, the meadow came alive with their collaborative efforts. The sun shone brighter, the wind rustled the leaves, and the air buzzed with a sense of purpose.

Pip and the squirrel, their bond stronger than ever, realized that even the smallest of creatures could achieve extraordinary things when they worked together. And so, their tale of teamwork was whispered throughout the meadow, inspiring other timid creatures to believe in their own potential.

Pip and the squirrel continued their small but mighty tasks, their laughter echoing through the meadow. The sun beat down on their fur, but their spirits remained warm, their determination as strong as the oak trees that stood tall around them.

As they worked, Pip noticed that the squirrels were struggling to reach the highest branches. With a smile, he hopped down and gently nudged them with his tiny nose. The squirrels looked up, surprised by the bunny's kindness.

"Thank you," said one squirrel, her voice soft but grateful. "We couldn't have reached those leaves without you."

Pip nodded, his heart filled with joy. He had helped his friends, and he had discovered a new sense of purpose.

Together, Pip and the squirrel continued their journey, their bond growing stronger with each passing day. They learned that even the smallest of creatures could achieve extraordinary things when they worked together, and that the most important thing was to believe in oneself and the power of friendship.

Pip and the squirrel continued their journey, their bond growing stronger with each passing day. They learned that even the smallest of creatures could achieve extraordinary things when they worked together, and that the most important thing was to believe in oneself and the power of friendship.

One day, they stumbled upon a group of butterflies struggling to navigate the maze-like meadow. Pip and the squirrel, with their combined strength and determination, helped them find their way out.

As they flew higher and higher, the meadow spread out beneath them like a breathtaking tapestry. Pip and the squirrel felt a sense of pride and accomplishment wash over them.

They realized that their teamwork had not only helped the butterflies but had also strengthened their own bond. They had learned that even the smallest of creatures could make a big difference when they worked together, and that the most important thing was to believe in themselves and the power of friendship.

Pip and the squirrel continued their adventures, their laughter echoing through the meadow. They knew that they had achieved something truly special, and they were grateful for the opportunity to have worked together.

Pip and the squirrel continued their adventures, their laughter echoing through the meadow. They knew that they had achieved something truly special, and they were grateful for the opportunity to have worked together.

Their journey led them to a hidden grove of wildflowers where they discovered a magical elixir that granted them the ability to grow taller and stronger. With newfound confidence, they soared high above the meadow, their laughter filling the air.

As they flew, they noticed a group of birds struggling to build their nests. Pip and the squirrel, with their combined strength and wisdom, helped them find the best materials and build the most sturdy nests.

The birds were amazed by their kindness and offered their thanks. Pip and the squirrel felt a sense of purpose and fulfillment knowing that they had made a difference in the lives of others.

They continued their adventures, exploring new and exciting places together. They learned that even the smallest of creatures could achieve extraordinary things when they worked together, and that the most important thing was to believe in themselves and the power of friendship.

.